In [ ]:
%load_ext autoreload
%autoreload 2

# Add/update embedding vectors for morphologies 

##  (or any type of KG resources, apart from ontology classes)


When adding a new set of embedding vectors or updating them, we need to perform the following sequence of steps

1. Given a model id, its revision, and a given a set of resources, ask the service[or some python code] for embedding vectors
2. Create/update embedding resources according to [this mapping](https://bbpgitlab.epfl.ch/dke/users/eugeniashurko/dataset-embeddings/-/blob/master/mappings/seu-embedding.hjson) --> model revision needs to be added to the `generation.activity.used.id`
3. Push them to Nexus
4. Tag them with the model UUID and the its revision (e.g. `e2b953b9-6724-4278-a1e5-3472bd63e374?rev=1`)

Related JIRA tickets: 
* https://bbpteam.epfl.ch/project/issues/browse/DKE-718
* https://bbpteam.epfl.ch/project/issues/browse/DKE-715

Prerequisites:

- The embedding model has been built
- Embedding service can read models from a dedicated Nexus project where all models are stored (here, at the moment, we can download models locally and get vectors directly from the models, without using the service)
- Model ID equals the Nexus resource id of the EmbeddingModel resource
- __Important__: local contexts in the projects with vectors should contain:

```
{
      "embedding": {
        "@id": "nsg:embedding",
        "@container": "@list"
      }
}
```

Questions:

* do we really need to url-encode tags ?
* add missing types and properties to the context

---

## Setup

### Imports

In [ ]:
import requests
import getpass

from kgforge.core import KnowledgeGraphForge

from inference_tools.similarity.data_registration import (create_forge_session,
                                                          load_embedding_models,
                                                          push_embedding_vectors)

In [ ]:
from kgforge.version import __version__
print(__version__)

---

## User input

In [ ]:
CONFIG_PATH = "../../../configs/new-forge-config.yaml"
ENDPOINT = "https://bbp.epfl.ch/nexus/v1"
DOWNLOAD_DIR = "../../../data"
TOKEN = getpass.getpass()

Bucket where embedding models live

In [ ]:
MODEL_CATALOG_ORG = "dke"
MODEL_CATALOG_PROJECT = "embedding-pipelines"

ID of the embedding model to use.

__PROVIDE HERE THE IDs OF YOUR MODELs (OPTIONAL, REVISION)__

In [ ]:
MODEL_IDS = [
    "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/d0c21fd5-cb9c-445c-b0a4-94847ba61f5a",  # neurite features
    "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/9fe6873b-ef6a-41b5-854a-382bc1be9fff",  # dendrite
    "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/84519407-ad30-4d31-877e-1d6560325393",  # axon
    "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/1c4fcd2e-000f-437b-b65b-844ee211105a",  # brain regions
    "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/608fab85-0cc9-4ff9-a4bd-4249589b5889",  # coordinates
    "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/43965be4-72f9-4901-9a95-d9ca13da8fb4",  # TMD
    "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/7a111efa-7467-42d2-9e0c-c1ca7a883216",  # TMD (scaled)
]

In [ ]:
MODEL_REVISIONS = {}  # Specify a revision (key model_id, value revision number), if necessary.
# If not specified the latest revision is used

Buckets where the input data lives together with the Bucket where the new embedding vectors should be registered.

In [ ]:
DATA_BUCKETS = {
    ("https://bbp.epfl.ch/nexus/v1", "bbp-external", "seu"):
         (
            "https://bbp.epfl.ch/nexus/v1",
             "dke",
             "seu-embeddings"
         )
}

If the embedding endpoint/bucket are not specified, we assume that embeddings should live in the same bucket as the input data.

---

## Create Forge sessions

### Session for embedding models

In [ ]:
forge_models = create_forge_session(
    CONFIG_PATH,
    (ENDPOINT, MODEL_CATALOG_ORG, MODEL_CATALOG_PROJECT),
    TOKEN)

### Sessions for different buckets for data and embedding vectors

In [ ]:
# TODO: find a way to pass different tokens and different configs
FORGE_SESSIONS = {}
for data_bucket, emb_bucket in DATA_BUCKETS.items():
    if data_bucket not in FORGE_SESSIONS:
        FORGE_SESSIONS[data_bucket] = create_forge_session(CONFIG_PATH, data_bucket, TOKEN)
    if emb_bucket not in FORGE_SESSIONS:
        FORGE_SESSIONS[emb_bucket] = create_forge_session(CONFIG_PATH, emb_bucket, TOKEN)

---

## Fetch resources from data buckets

Data type filter for generating embedding vectors

In [ ]:
DATA_TYPE_FILTER = "NeuronMorphology"
HARD_RESOURCE_LIMIT = 10000  # Limit on number of resources we can retrieve with SPARQL queries

In [ ]:
resource_set = {}
for bucket_config in DATA_BUCKETS.keys():
    if bucket_config not in resource_set:
        forge = FORGE_SESSIONS[bucket_config]
        query = f"""
            SELECT ?id
            WHERE {{
                ?id a {DATA_TYPE_FILTER} ;
                    <https://bluebrain.github.io/nexus/vocabulary/deprecated> false .
            }}
        """ 
        resources = forge.sparql(query, limit=HARD_RESOURCE_LIMIT)
        resources = [forge.retrieve(r.id) for r in resources] 

        resource_set[bucket_config] = resources

In [ ]:
for k, v in resource_set.items():
    print("Bucket: ", k)
    print("\t", len(v), "resources")
    print()

## Load the embedding model

In [ ]:
model_revisions, model_tags, pipelines = load_embedding_models(
    forge_models, MODEL_IDS, model_revisions=MODEL_REVISIONS, dowload_dir=DOWNLOAD_DIR)

## Compute embedding vectors for all the resources and push to Nexus

- TODO: add the NeuronMorphology revision once available
- TODO: add prediction of previously unseen points (currently, only the in-sample points are considered)

In [ ]:
SEU_DICTIONARY_MAPPING = "../../../mappings/seu-embedding.hjson"

In [ ]:
push_embedding_vectors(
    FORGE_SESSIONS, DATA_BUCKETS, MODEL_IDS, model_revisions, model_tags,
    pipelines, resource_set, SEU_DICTIONARY_MAPPING)

The following tag should be used to create new ES views on the vectors.

In [ ]:
model_tags